■ **old man and the sea**의 전체 내용을 학습하고, **"How old was I when you first took me in a boat?" 이후의 10개의 token**을 생성하시오.
- model은 이전 **token 30개로 다음 1개의 token을 예측**하도록 한다.
- model은 **LSTM layer 2개를 포함**한다.
- **epoch은 20회로 설정**하며, 나머지 파라미터는 임의로 설정한다. (epoch은 약 1000회 정도 되어야 좋은 결과를 보일 수 있으나, 학습 시간을 줄이기 위해 20으로 설정함, 결과가 좋지 않음을 감안할 것)
- 해당 문장 뒤의 **실제 token 10개**와 model이 **예측한 10개의 token**을 비교하시오. (725번째(index=724) token부터 10개의 token 확인)
- <strong><font color='red' style='bold'>주석을 이용하여 각 과정의 역할과 변수의 의미를 상세히 서술하시오.</font></strong>

In [2]:
def read_file(filepath):
    with open(filepath) as f:
        text = f.read()
    return text

In [3]:
read_file('old_man_and_the_sea.txt')

'He was an old man who fished alone in a skiff in the Gulf Stream and he had gone eighty-four days now without taking a fish. In the first forty days a boy had been with him. But after forty days without a fish the boy\'s parents had told him that the old man was now definitely and finally salao, which is the worst form of unlucky, and the boy had gone at their orders in another boat which caught three good fish the first week. It made the boy sad to see the old man come in each day with his skiff empty and he always went down to help him carry either the coiled lines or the gaff and harpoon and the sail that was furled around the mast. The sail was patched with flour sacks and, furled, it looked like the flag of permanent defeat.\n\nThe old man was thin and gaunt with deep wrinkles in the back of his neck. The brown blotches of the benevolent skin cancer the sun brings from its reflection on the tropic sea were on his cheeks. The blotches ran well down the sides of his face and his ha

In [4]:
import spacy

nlp = spacy.load('en_core_web_md',disable=['parser','tagger','ner'])
# 필요없는 과정 생략, tokenizer가 조금 더 빨라짐

In [5]:
# 최대 길이 설정 
nlp.max_length = len(read_file('old_man_and_the_sea.txt'))

In [6]:
# 나열된 문자열에 포함된 text가 아닐 경우, 소문자 형태로 변환하는 함수
def separate_punc(doc_text):
    return [token.text.lower() for token in nlp(doc_text) if token.text not in '\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n ']

In [7]:
text = read_file('old_man_and_the_sea.txt')
tokens = separate_punc(text)
# parser, tagger, ner을 pipeline에서 생략하여 warning 발생

C:\ProgramData\anaconda3\Lib\site-packages\spacy\pipeline\lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


In [8]:
tokens

['he',
 'was',
 'an',
 'old',
 'man',
 'who',
 'fished',
 'alone',
 'in',
 'a',
 'skiff',
 'in',
 'the',
 'gulf',
 'stream',
 'and',
 'he',
 'had',
 'gone',
 'eighty',
 'four',
 'days',
 'now',
 'without',
 'taking',
 'a',
 'fish',
 'in',
 'the',
 'first',
 'forty',
 'days',
 'a',
 'boy',
 'had',
 'been',
 'with',
 'him',
 'but',
 'after',
 'forty',
 'days',
 'without',
 'a',
 'fish',
 'the',
 'boy',
 "'s",
 'parents',
 'had',
 'told',
 'him',
 'that',
 'the',
 'old',
 'man',
 'was',
 'now',
 'definitely',
 'and',
 'finally',
 'salao',
 'which',
 'is',
 'the',
 'worst',
 'form',
 'of',
 'unlucky',
 'and',
 'the',
 'boy',
 'had',
 'gone',
 'at',
 'their',
 'orders',
 'in',
 'another',
 'boat',
 'which',
 'caught',
 'three',
 'good',
 'fish',
 'the',
 'first',
 'week',
 'it',
 'made',
 'the',
 'boy',
 'sad',
 'to',
 'see',
 'the',
 'old',
 'man',
 'come',
 'in',
 'each',
 'day',
 'with',
 'his',
 'skiff',
 'empty',
 'and',
 'he',
 'always',
 'went',
 'down',
 'to',
 'help',
 'him',
 'car

In [9]:
len(tokens)

6875

In [28]:
# n개의 연속된 token으로 하나의 token을 예측하도록 함
# sequence의 길이: n + 1
# n개의 token: X, 마지막 1개의 token: y
pre_tokens = 30
seq_len = pre_tokens+1

text_sequences = []

for i in range(seq_len,len(tokens)):
    seq = tokens[i-seq_len:i]
    text_sequences.append(seq)

In [29]:
text_sequences[0]

['he',
 'was',
 'an',
 'old',
 'man',
 'who',
 'fished',
 'alone',
 'in',
 'a',
 'skiff',
 'in',
 'the',
 'gulf',
 'stream',
 'and',
 'he',
 'had',
 'gone',
 'eighty',
 'four',
 'days',
 'now',
 'without',
 'taking',
 'a',
 'fish',
 'in',
 'the',
 'first',
 'forty']

In [30]:
' '.join(text_sequences[100])

'each day with his skiff empty and he always went down to help him carry either the coiled lines or the gaff and harpoon and the sail that was furled around'

In [31]:
len(text_sequences)
# len(tokens) - seq_len

6844

In [32]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)
sequences = tokenizer.texts_to_sequences(text_sequences)

In [33]:
sequences[0]

[3,
 6,
 237,
 10,
 12,
 107,
 399,
 398,
 8,
 7,
 121,
 8,
 1,
 586,
 585,
 2,
 3,
 24,
 236,
 162,
 396,
 192,
 32,
 191,
 584,
 7,
 23,
 8,
 1,
 129,
 238]

In [34]:
tokenizer.index_word
# word와 정수의 pair (dict)

{1: 'the',
 2: 'and',
 3: 'he',
 4: 'of',
 5: 'i',
 6: 'was',
 7: 'a',
 8: 'in',
 9: 'to',
 10: 'old',
 11: 'it',
 12: 'man',
 13: 'you',
 14: 'that',
 15: 'his',
 16: 'boy',
 17: 'said',
 18: 'they',
 19: 'but',
 20: 'were',
 21: 'on',
 22: 'as',
 23: 'fish',
 24: 'had',
 25: 'with',
 26: 'them',
 27: 'for',
 28: 'him',
 29: 'when',
 30: 'is',
 31: 'not',
 32: 'now',
 33: 'out',
 34: 'water',
 35: 'have',
 36: 'then',
 37: 'can',
 38: 'at',
 39: 'no',
 40: 'there',
 41: 'so',
 42: 'from',
 43: 'are',
 44: 'their',
 45: 'be',
 46: 'would',
 47: 'we',
 48: 'great',
 49: 'down',
 50: 'too',
 51: "'s",
 52: 'big',
 53: 'me',
 54: 'into',
 55: 'all',
 56: 'one',
 57: 'do',
 58: 'like',
 59: 'where',
 60: 'sun',
 61: 'back',
 62: 'made',
 63: 'or',
 64: 'did',
 65: 'thought',
 66: 'see',
 67: 'went',
 68: 'eyes',
 69: 'up',
 70: 'will',
 71: 'good',
 72: 'two',
 73: 'could',
 74: 'know',
 75: 'think',
 76: 'boat',
 77: 'come',
 78: 'day',
 79: 'sea',
 80: "'ll",
 81: 'take',
 82: 'fishermen

In [35]:
len(tokenizer.index_word)

1276

In [36]:
for i in sequences[0]:
    print(f'{i}:{tokenizer.index_word[i]}')

3:he
6:was
237:an
10:old
12:man
107:who
399:fished
398:alone
8:in
7:a
121:skiff
8:in
1:the
586:gulf
585:stream
2:and
3:he
24:had
236:gone
162:eighty
396:four
192:days
32:now
191:without
584:taking
7:a
23:fish
8:in
1:the
129:first
238:forty


In [37]:
tokenizer.word_counts
# 각 단어의 빈도수

OrderedDict([('he', 5661),
             ('was', 3654),
             ('an', 127),
             ('old', 2907),
             ('man', 2752),
             ('who', 285),
             ('fished', 69),
             ('alone', 70),
             ('in', 3273),
             ('a', 3291),
             ('skiff', 259),
             ('the', 20511),
             ('gulf', 45),
             ('stream', 46),
             ('and', 9596),
             ('had', 1413),
             ('gone', 143),
             ('eighty', 206),
             ('four', 83),
             ('days', 177),
             ('now', 1007),
             ('without', 179),
             ('taking', 56),
             ('fish', 1546),
             ('first', 247),
             ('forty', 124),
             ('boy', 2015),
             ('been', 186),
             ('with', 1364),
             ('him', 1302),
             ('but', 1643),
             ('after', 279),
             ("'s", 620),
             ('parents', 31),
             ('told', 124),
             (

In [38]:
vocab_size = len(tokenizer.word_counts)

In [39]:
vocab_size

1276

In [40]:
import numpy as np

In [41]:
sequences = np.array(sequences)

In [42]:
sequences

array([[  3,   6, 237, ...,   1, 129, 238],
       [  6, 237,  10, ..., 129, 238, 192],
       [237,  10,  12, ..., 238, 192,   7],
       ...,
       [ 26,  27, 142, ...,  49,  15,  61],
       [ 27, 142,  20, ...,  15,  61,  22],
       [142,  20,  49, ...,  61,  22,   3]])

In [43]:
sequences.shape
# X: column index 0~24 , y: column index 25

(6844, 31)

In [44]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding

In [45]:
# model에 LSTM layer 2개를 포함

def create_model(dict_size,sequence_len,embbeding_dim):
    model = Sequential()
    # Embedding layer
    model.add(Embedding(dict_size,embbeding_dim,input_length=sequence_len))
    # 1st LSTM
    model.add(LSTM(150,return_sequences=True)),
    # 2nd LSTM
    model.add(LSTM(150)),
    # 1st Dense
    model.add(Dense(150,activation='tanh')),
    # 2nd Dense (ouyput layer, class의 수 = vocabulary의 수)
    model.add(Dense(dict_size,activation='softmax'))
    
    model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])
    model.summary()
    return model

In [46]:
from keras.utils import to_categorical

In [47]:
print(sequences.shape)

(6844, 31)


In [48]:
# 첫 30개의 words (X, 입력)
X = sequences[:,:-1]

In [49]:
# 마지막 단어 (y, target)
y = sequences[:,-1]

In [50]:
y = to_categorical(y,num_classes=vocab_size+1)

In [51]:
print(y.shape)
# y.shape[1]: class의 수, 단어의 수 + 1

(6844, 1277)


In [52]:
seq_len = X.shape[1]
# X.shape[1]: 다음 단어를 예측하기 위해 입력되는 단어의 수

In [53]:
model = create_model(vocab_size+1,seq_len,16)



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 30, 16)            20432     
                                                                 
 lstm (LSTM)                 (None, 30, 150)           100200    
                                                                 
 lstm_1 (LSTM)               (None, 150)               180600    
                                                                 
 dense (Dense)               (None, 150)               22650     
                                                                 
 dense_1 (Dense)             (None, 1277)              192827    
                                                                 
Total params: 516709 (1.97 MB)
Trainable params: 516709 (1.97 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [54]:
from pickle import dump, load

In [55]:
# epoch 20회 설정

learning_result = model.fit(X,y,batch_size=256,epochs=20)

Epoch 1/20


27/27 [==============================] - 5s 75ms/step - loss: 6.3752 - acc: 0.0934
Epoch 2/20
27/27 [==============================] - 2s 62ms/step - loss: 5.7648 - acc: 0.0967
Epoch 3/20
27/27 [==============================] - 2s 60ms/step - loss: 5.7268 - acc: 0.0967
Epoch 4/20
27/27 [==============================] - 2s 59ms/step - loss: 5.7132 - acc: 0.0967
Epoch 5/20
27/27 [==============================] - 2s 60ms/step - loss: 5.7090 - acc: 0.0967
Epoch 6/20
27/27 [==============================] - 2s 60ms/step - loss: 5.7054 - acc: 0.0967
Epoch 7/20
27/27 [==============================] - 2s 58ms/step - loss: 5.7048 - acc: 0.0967
Epoch 8/20
27/27 [==============================] - 2s 60ms/step - loss: 5.7050 - acc: 0.0967
Epoch 9/20
27/27 [==============================] - 2s 60ms/step - loss: 5.7015 - acc: 0.0967
Epoch 10/20
27/27 [==============================] - 2s 59ms/step - loss: 5.7039 - acc: 0.0967
Epoch 11/20
27/27 [==============================] - 2s 6

In [56]:
model.save('text_generation.keras')
dump(tokenizer,open('text_generation','wb'))

In [58]:
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

In [59]:
sample1 = 'How old was I when you first took me in a boat?'

enc = tokenizer.texts_to_sequences([sample1])
pad = pad_sequences(enc,maxlen=3,truncating='pre', padding='pre')
ind = model.predict(sequences[200:201,:-1])[0].argmax()
tokenizer.index_word[ind]

1/1 [==============================] - 1s 575ms/step


'old'

In [60]:
def generate_text(model,tokenizer,seq_len,input_text,num_gen_words):
    # model: 학습한 model
    # tokenizer: text를 int로 변환하기 위한 사전
    # seq_len: sequence의 길이
    # input_text: text input(raw text, string)
    # num_gen_words: model에 의해 생성되는 word의 수
    
    # output
    output_text = []
    
    # Create num_gen_words
    for i in range(num_gen_words):
        # 1. encoding
        encoded_text = tokenizer.texts_to_sequences([input_text])
        # texts_to_sequences는 여러 개의 text sequence가 입력된다고 가정함
        # keras의 함수나 method는 일반적으로 2D array, 2D list를 입력으로 한다.
        # 하나의 string을 포함한 list를 texts_to_sequence 입력으로 하면
        # 출력은 2D list가 된다. (위의 cell 참고)
        # encoded_text는 정수 sequence로 변환된 input_text
        
        # 2. padding
        pad_encoded = pad_sequences(encoded_text,maxlen=seq_len,truncating='pre')
        # encoded_text: integer sequence
        # max_len: sequence의 길이, 더 짧은 경우, padding(0으로 채움), 더 긴 경우, 잘라냄
        # truncating='pre': 길이가 max_len보다 긴 문장일 경우, 문장의 앞을 잘라냄
        # encoded_text가 하나의 sample이라도 출력은 2d array가 반환됨
        # 위의 cell 참고
        
        # 3. predicting
        pred_word_ind = model.predict(pad_encoded,verbose=0).argmax()
        # model.predict의 결과는 tokenizer가 포함하고 있는 word에 대해
        # 각 word일 확률을 반환함 (하나의 sample(문장)에 대해 vocab_size 만큼의 결과를 반환)
        # 그러므로 predict가 반환한 결과에서 최대값(최대확률)을 갖는 column의 index가 필요하다.
        # verbose=0으로 설정하면 predict과정에 대해 출력하지 않는다.
        
        # index를 단어로 변환
        pred_word = tokenizer.index_word[pred_word_ind]
        
        # 문장을 update
        # input_text 뒤에 예측된 결과를 연결하면, padding 과정에서 문장앞의 단어가 하나씩 제외된다.
        input_text += ' ' + pred_word
        # 예측된 결과 단어를 output_text에 연결함
        output_text.append(pred_word)
        
    return ' '.join(output_text)

In [61]:
input_ind = 0
seed_text = text_sequences[input_ind]
seed_text = ' '.join(seed_text)
seed_text

'he was an old man who fished alone in a skiff in the gulf stream and he had gone eighty four days now without taking a fish in the first forty'

In [62]:
output_text = generate_text(model,tokenizer,10,seed_text,10)

In [63]:
print(' '.join(text_sequences[input_ind+26]))
print(output_text)
# 실제와 예측 token 비교

fish in the first forty days a boy had been with him but after forty days without a fish the boy 's parents had told him that the old man was
and the old the old the old the old the
